In [22]:
import pandas as pd
from pprint import pprint


df = pd.read_json('./archive/dataset/cpu.json')

print(df.to_string())

                                                   name    price  core_count  core_clock  boost_clock  tdp                 graphics    smt
0                                     AMD Ryzen 5 5600X   168.99           6       3.700          4.6   65                     None   True
1                                     AMD Ryzen 7 5800X   228.72           8       3.800          4.7  105                     None   True
2                                  Intel Core i7-13700K   381.99          16       3.400          5.4  125   Intel UHD Graphics 770   True
3                                  Intel Core i9-13900K   568.99          24       3.000          5.8  125   Intel UHD Graphics 770   True
4                                   AMD Ryzen 7 7800X3D   439.00           8       4.200          5.0  120                   Radeon   True
5                                  Intel Core i5-13600K   308.95          14       3.500          5.1  125   Intel UHD Graphics 770   True
6                          

In [23]:
cpu_list = pd.DataFrame(df).dropna().to_dict(orient="records")
pprint(cpu_list)

[{'boost_clock': 5.4,
  'core_clock': 3.4,
  'core_count': 16,
  'graphics': 'Intel UHD Graphics 770',
  'name': 'Intel Core i7-13700K',
  'price': 381.99,
  'smt': True,
  'tdp': 125},
 {'boost_clock': 5.8,
  'core_clock': 3.0,
  'core_count': 24,
  'graphics': 'Intel UHD Graphics 770',
  'name': 'Intel Core i9-13900K',
  'price': 568.99,
  'smt': True,
  'tdp': 125},
 {'boost_clock': 5.0,
  'core_clock': 4.2,
  'core_count': 8,
  'graphics': 'Radeon',
  'name': 'AMD Ryzen 7 7800X3D',
  'price': 439.0,
  'smt': True,
  'tdp': 120},
 {'boost_clock': 5.1,
  'core_clock': 3.5,
  'core_count': 14,
  'graphics': 'Intel UHD Graphics 770',
  'name': 'Intel Core i5-13600K',
  'price': 308.95,
  'smt': True,
  'tdp': 125},
 {'boost_clock': 5.4,
  'core_clock': 4.5,
  'core_count': 8,
  'graphics': 'Radeon',
  'name': 'AMD Ryzen 7 7700X',
  'price': 299.0,
  'smt': True,
  'tdp': 105},
 {'boost_clock': 5.3,
  'core_clock': 4.7,
  'core_count': 6,
  'graphics': 'Radeon',
  'name': 'AMD Ryzen 5 7

In [25]:
def filter_by_brand(cpu_catalog):
    filter_list = []
    remaining_items = []
    search_str = input("please enter an input: ")
    print(len(cpu_catalog))
    for item in cpu_catalog:
        if search_str.lower() in item["name"].lower():
            filter_list.append(item)
        else:
            remaining_items.append(item)
    return filter_list
pprint(filter_by_brand(cpu_list))
results = filter_by_brand(cpu_list)

156
[{'boost_clock': 5.0,
  'core_clock': 4.2,
  'core_count': 8,
  'graphics': 'Radeon',
  'name': 'AMD Ryzen 7 7800X3D',
  'price': 439.0,
  'smt': True,
  'tdp': 120},
 {'boost_clock': 5.4,
  'core_clock': 4.5,
  'core_count': 8,
  'graphics': 'Radeon',
  'name': 'AMD Ryzen 7 7700X',
  'price': 299.0,
  'smt': True,
  'tdp': 105},
 {'boost_clock': 5.3,
  'core_clock': 4.7,
  'core_count': 6,
  'graphics': 'Radeon',
  'name': 'AMD Ryzen 5 7600X',
  'price': 229.0,
  'smt': True,
  'tdp': 105},
 {'boost_clock': 5.1,
  'core_clock': 3.8,
  'core_count': 6,
  'graphics': 'Radeon',
  'name': 'AMD Ryzen 5 7600',
  'price': 212.52,
  'smt': True,
  'tdp': 65},
 {'boost_clock': 4.4,
  'core_clock': 3.9,
  'core_count': 6,
  'graphics': 'Radeon Vega 7',
  'name': 'AMD Ryzen 5 5600G',
  'price': 139.0,
  'smt': True,
  'tdp': 65},
 {'boost_clock': 5.6,
  'core_clock': 4.7,
  'core_count': 12,
  'graphics': 'Radeon',
  'name': 'AMD Ryzen 9 7900X',
  'price': 419.99,
  'smt': True,
  'tdp': 170

In [ ]:
# def filter_by_graphics(dictionary):
#     isgpu_list = []
#     nogpu_list = []
#     for gpu in dictionary: 
#         gpu_status = gpu.get("graphics")
#         if gpu_status is not None:
#             isgpu_list.append(gpu)
#         else:
#             nogpu_list.append(gpu) 

#     pprint(isgpu_list)
#     return isgpu_list

# gpu_list = filter_by_graphics(results)

[{'boost_clock': 5.0,
  'core_clock': 4.2,
  'core_count': 8,
  'graphics': 'Radeon',
  'name': 'AMD Ryzen 7 7800X3D',
  'price': 439.0,
  'smt': True,
  'tdp': 120},
 {'boost_clock': 5.4,
  'core_clock': 4.5,
  'core_count': 8,
  'graphics': 'Radeon',
  'name': 'AMD Ryzen 7 7700X',
  'price': 299.0,
  'smt': True,
  'tdp': 105},
 {'boost_clock': 5.3,
  'core_clock': 4.7,
  'core_count': 6,
  'graphics': 'Radeon',
  'name': 'AMD Ryzen 5 7600X',
  'price': 229.0,
  'smt': True,
  'tdp': 105},
 {'boost_clock': 5.1,
  'core_clock': 3.8,
  'core_count': 6,
  'graphics': 'Radeon',
  'name': 'AMD Ryzen 5 7600',
  'price': 212.52,
  'smt': True,
  'tdp': 65},
 {'boost_clock': 4.4,
  'core_clock': 3.9,
  'core_count': 6,
  'graphics': 'Radeon Vega 7',
  'name': 'AMD Ryzen 5 5600G',
  'price': 139.0,
  'smt': True,
  'tdp': 65},
 {'boost_clock': 5.6,
  'core_clock': 4.7,
  'core_count': 12,
  'graphics': 'Radeon',
  'name': 'AMD Ryzen 9 7900X',
  'price': 419.99,
  'smt': True,
  'tdp': 170},
 